### Stock News Monitoring Project

1. Request stock data from an API, calculate stock prices. (Stock Price change, change %)

2.  Checking out the reason for this, i.e. News

3. Send all this in SMS, which will help in decision making to buy or sell the stock

In [4]:
'''
Digging documentation of API

INDENT : Tab
UNINDENT : Shift + Tab
'''

'\nDigging documentation of API\n\nINDENT : Tab\nUNINDENT : Shift + Tab\n'

#### Alphavantage API to get closing prices or Yahoo Finance/ yfinance lib to get the stock data

##### tapping into prices and calculation to show Stock price difference and change percentage

#### alphavantage quite hard to use

### News API to get the news about the stock
- Under everything endpoint

#### Then use Twilio SMS through Python

### Source Code👇

In [18]:
import requests, api_keys
from twilio.rest import Client

VIRTUAL_TWILIO_NUMBER = api_keys.MY_TWILIO_NUMBER
VERIFIED_NUMBER = api_keys.MY_NUMBER

STOCK_NAME = "MSFT"
COMPANY_NAME = "Microsoft Corporation"

STOCK_ENDPOINT = "https://www.alphavantage.co/query"
NEWS_ENDPOINT = "https://newsapi.org/v2/everything"

STOCK_API_KEY = api_keys.ALPHAVANTAGE_API_KEY
NEWS_API_KEY = api_keys.NEWS_API_KEY
TWILIO_SID = api_keys.TWILIO_ACCOUNT_SID
TWILIO_AUTH_TOKEN = api_keys.TWILIO_AUTH_TOKEN

## STEP 1: Use https://www.alphavantage.co/documentation/#daily
# When stock price increase/decreases by 5% between yesterday and the day before yesterday then print("Get News").

#Get yesterday's closing stock price
stock_params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": STOCK_NAME,
    "apikey": STOCK_API_KEY,
}

response = requests.get(STOCK_ENDPOINT, params=stock_params)

data = response.json()["Time Series (Daily)"]
data_list = [value for (key, value) in data.items()]
yesterday_data = data_list[0]
yesterday_closing_price = yesterday_data["4. close"]
print(yesterday_closing_price)

#Get the day before yesterday's closing stock price
day_before_yesterday_data = data_list[1]
day_before_yesterday_closing_price = day_before_yesterday_data["4. close"]
print(day_before_yesterday_closing_price)

#Find the positive difference between 1 and 2. e.g. 40 - 20 = -20, but the positive difference is 20. Hint: https://www.w3schools.com/python/ref_func_abs.asp
difference = float(yesterday_closing_price) - float(day_before_yesterday_closing_price)
up_down = None
if difference > 0:
    up_down = "🔺"
else:
    up_down = "🔻"

#Work out the percentage difference in price between closing price yesterday and closing price the day before yesterday.
diff_percent = round((difference / float(yesterday_closing_price)) * 100)
print(diff_percent)


    ## STEP 2: Instead of printing ("Get News"), actually get the first 3 news pieces for the COMPANY_NAME.

#Instead of printing ("Get News"), use the News API to get articles related to the COMPANY_NAME.
#If difference percentage is greater than 5 then print("Get News").
if abs(diff_percent) > 1:
    news_params = {
        "apiKey": NEWS_API_KEY,
        "qInTitle": COMPANY_NAME,
    }

    news_response = requests.get(NEWS_ENDPOINT, params=news_params)
    articles = news_response.json()["articles"]

    #Use Python slice operator to create a list that contains the first 3 articles. Hint: https://stackoverflow.com/questions/509211/understanding-slice-notation
    three_articles = articles[:3]
    print(three_articles)

    ## STEP 3: Use Twilio to send a seperate message with each article's title and description to your phone number.

    #Create a new list of the first 3 article's headline and description using list comprehension.
    formatted_articles = [f"{STOCK_NAME}: {up_down}{diff_percent}%\nHeadline: {article['title']}. \nBrief: {article['description']}" for article in three_articles]
    print(formatted_articles)
    #Send each article as a separate message via Twilio.
    client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

    #TODO 8. - Send each article as a separate message via Twilio.
    for article in formatted_articles:
        message = client.messages.create(
            body=article,
            from_=VIRTUAL_TWILIO_NUMBER,
            to=VERIFIED_NUMBER
        )


240.9800
238.9500
1
[{'source': {'id': None, 'name': 'Seeking Alpha'}, 'author': 'SA Transcripts', 'title': 'Microsoft Corporation (MSFT) Presents at Deutsche Bank 2022 Technology Conference (Transcript)', 'description': "Microsoft Corporation (NASDAQ:NASDAQ:MSFT) Deutsche Bank 2022 Technology Conference August 31, 2022 1:30 PM ETCompany ParticipantsDave O'Hara - Executive Vice President & Chief...", 'url': 'https://seekingalpha.com/article/4538257-microsoft-corporation-msft-presents-deutsche-bank-2022-technology-conference-transcript', 'urlToImage': 'https://static.seekingalpha.com/assets/og_image_1200-29b2bfe1a595477db6826bd2126c63ac2091efb7ec76347a8e7f81ba17e3de6c.png', 'publishedAt': '2022-08-31T20:42:05Z', 'content': "Microsoft Corporation (NASDAQ:MSFT) Deutsche Bank 2022 Technology Conference August 31, 2022 1:30 PM ET\r\n Company Participants\r\n Dave O'Hara - Executive Vice President &amp; Chief Financial Officer, … [+37079 chars]"}, {'source': {'id': None, 'name': 'Yahoo Enter